In [22]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [37]:
import keras
import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model, create_and_compile_model_exog

series = pd.DataFrame(
    {
        "1": pd.Series(np.arange(10.)),
        "2": pd.Series(np.arange(50., 60.)),
        "3": pd.Series(np.arange(100., 110.)),
    }
)

exog = pd.DataFrame(
    {
        "exog_1": pd.Series(np.arange(50.)),
        "exog_2": pd.Series(np.arange(50., 100.)),
        "exog_3": pd.Series(np.arange(100., 150.)),
    }
)

lags = 3
steps = 2
levels = ["1", "2"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]


model = create_and_compile_model(
    series=series,
    lags=lags,
    steps=steps,
    levels=levels,
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

model = create_and_compile_model_exog(
    series=series,
    lags=lags,
    steps=steps,
    levels=levels,
    exog=exog,
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0


TypeError: object of type 'int' has no len()

In [31]:
forecaster = ForecasterRnn(model, levels, lags=lags)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
forecaster.fit(series)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step - loss: 0.4868


In [18]:
_ = forecaster.create_train_X_y(series)
_[0].shape

(44, 3, 3)

In [35]:
forecaster.in_sample_residuals_[1]

array([[0.30619231, 0.30203365],
       [0.41336301, 0.40936604],
       [0.52067263, 0.51567958],
       [0.62780604, 0.62108079],
       [0.73467941, 0.72591045],
       [0.84144858, 0.83003627]])

In [16]:
forecaster.in_sample_residuals_[2][:5, :]

array([[-0.03076794, -0.08773223],
       [-0.01476122, -0.07540671],
       [ 0.00093351, -0.06347512],
       [ 0.0164708 , -0.05176229],
       [ 0.03186672, -0.04018809]])

In [5]:
forecaster.series_names_in_

['1', '2', '3']

In [6]:
forecaster.levels

['1', '2']

In [7]:
forecaster.last_window_

,1,2,3
47,47.0,97.0,147.0
48,48.0,98.0,148.0
49,49.0,99.0,149.0


In [8]:
inputs = forecaster._create_predict_inputs(steps=3, levels=['1', '2'])
inputs

(array([[[0.95918367, 0.95918367, 0.95918367],
         [0.97959184, 0.97959184, 0.97959184],
         [1.        , 1.        , 1.        ]]]),
 [],
 [1, 2, 3],
 ['1', '2'],
 Index([50, 51, 52], dtype='int64'))

In [36]:
predictions = forecaster.predict(steps=2, levels=['1', "2"])
predictions

,level,pred
10,1,0.512350
10,2,50.650616
11,1,0.694859
11,2,50.684875


In [34]:
_[0]

array([[[0.        , 0.        , 0.        ],
        [0.11111111, 0.11111111, 0.11111111],
        [0.22222222, 0.22222222, 0.22222222]],

       [[0.11111111, 0.11111111, 0.11111111],
        [0.22222222, 0.22222222, 0.22222222],
        [0.33333333, 0.33333333, 0.33333333]],

       [[0.22222222, 0.22222222, 0.22222222],
        [0.33333333, 0.33333333, 0.33333333],
        [0.44444444, 0.44444444, 0.44444444]],

       [[0.33333333, 0.33333333, 0.33333333],
        [0.44444444, 0.44444444, 0.44444444],
        [0.55555556, 0.55555556, 0.55555556]],

       [[0.44444444, 0.44444444, 0.44444444],
        [0.55555556, 0.55555556, 0.55555556],
        [0.66666667, 0.66666667, 0.66666667]],

       [[0.55555556, 0.55555556, 0.55555556],
        [0.66666667, 0.66666667, 0.66666667],
        [0.77777778, 0.77777778, 0.77777778]]])

In [ ]:
steps = 2
levels = 2

_ = forecaster.create_train_X_y(series)
forecaster.regressor.predict(x=_[0], verbose=0)

array([[[0.02714103, 0.03129969],
        [0.03067968, 0.02870654]],

       [[0.03108143, 0.03507841],
        [0.03619017, 0.03491943]],

       [[0.03488293, 0.03987598],
        [0.04253578, 0.04175308]],

       [[0.03886062, 0.04558587],
        [0.04964645, 0.04881068]],

       [[0.04309836, 0.05186733],
        [0.056449  , 0.05565476]],

       [[0.04744031, 0.05885262],
        [0.06324376, 0.06249271]]], dtype=float32)

In [ ]:
_ = forecaster.create_train_X_y(series)
forecaster.regressor.predict(x=_[0], verbose=0)

In [125]:
predictions = forecaster.predict(steps=3, levels=None)
predictions

,level,pred
50,1,7.713717
50,2,61.483234
51,1,25.941444
51,2,55.504932
52,1,31.411823
52,2,82.724594


In [9]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=None)
predictions

[[0.55023268 0.45024333]
 [0.58160202 0.35509963]
 [0.41151223 0.49386649]]


,level,pred,lower_bound,upper_bound
50,1,18.687877,-8.273525,45.649278
50,2,74.351726,52.289802,96.413649
51,1,18.113966,-10.384533,46.612465
51,2,80.556508,63.156627,97.956390
52,1,28.626856,8.462757,48.790955
52,2,74.064660,49.865202,98.264118


In [140]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=['1'])
predictions

,level,pred,lower_bound,upper_bound
50,1,7.713717,-29.086494,44.513928
51,1,25.941444,4.372634,47.510253
52,1,31.411823,13.783603,49.040043


In [144]:
np.quantile(
    np.abs(forecaster.in_sample_residuals_[2][:, 1]), 0.95
)

np.float64(0.8132576974450934)

In [ ]:
forecaster.in_sample_residuals_[1]

{1: array([[ 0.00300063, -0.02063499],
        [ 0.02164905, -0.00324193],
        [ 0.04019636,  0.01412207],
        [ 0.05871693,  0.03156052],
        [ 0.07703444,  0.04902489],
        [ 0.09533564,  0.06641259],
        [ 0.11370614,  0.0837122 ],
        [ 0.13206353,  0.10091294],
        [ 0.15043704,  0.11811714],
        [ 0.1688066 ,  0.1353427 ],
        [ 0.1871703 ,  0.15255236],
        [ 0.20552565,  0.16975107],
        [ 0.2238739 ,  0.18696467],
        [ 0.2422113 ,  0.20416079],
        [ 0.26053225,  0.22133179],
        [ 0.2789068 ,  0.23851671],
        [ 0.29727427,  0.2556899 ],
        [ 0.31563729,  0.27285653],
        [ 0.33399589,  0.29001659],
        [ 0.35235008,  0.30717007],
        [ 0.37069992,  0.32431521],
        [ 0.38904568,  0.34144994],
        [ 0.40738741,  0.3585762 ],
        [ 0.42566772,  0.37572525],
        [ 0.44391787,  0.39288245],
        [ 0.46216279,  0.41003336],
        [ 0.48040247,  0.42717802],
        [ 0.49863687,  0.